In [21]:
import pickle
import anndata2ri
import scanpy
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr

#This activates the object conversion.
anndata2ri.activate()
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


/tmp/ipykernel_2679499/19227154.py:9: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()


In [22]:
#Import needed R libraries.
seurat = importr('Seurat')
base = importr('base')

In [23]:
#Read Seurat object into R.
robjects.r('seurat_object = readRDS("/scratch/hnatri/PIPAC/nonimmune_complete_subset.rds")')

<rpy2.robjects.methods.RS4 object at 0x1554b3e22480> [RTYPES.S4SXP]
R classes: ('Seurat',)

In [24]:
#Merge layers because Seurat object versioning is wild. Convert Seurat object to Anndata.
robjects.r('seurat_object = JoinLayers(seurat_object)')
adata = robjects.r('as.SingleCellExperiment(seurat_object)')

In [25]:
#Check to make sure conversion worked and possibly cast counts to array if necessary.
adata.X = adata.X.toarray()

In [26]:
adata.X

array([[0.        , 0.        , 0.7412763 , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.52658904,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.52658904, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [27]:
del adata.layers

In [28]:
#Write anndata to pickle file. Next move to the clustering script.
with open("/scratch/hnatri/PIPAC/nonimmune_complete_subset", "wb") as adata_object:
    pickle.dump(adata, adata_object)

In [29]:
adata

AnnData object with n_obs × n_vars = 812247 × 480
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'cell_id', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'genomic_control_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'nucleus_count', 'segmentation_method', 'num.blank', 'TMA', 'percent.blank', 'nCount_cell_RNA', 'nFeature_cell_RNA', 'Sample', 'ident', 'leiden_res0.5', 'leiden_res1.0', 'leiden_res1.5', 'leiden_res2.0', 'Institution', 'Patient_ID', 'Timepoint', 'Tissue', 'Location_Quadrant', 'Lineage', 'annot1'

In [30]:
adata.obs["annot1"].unique()

array(['Epi2', 'Fibr2', 'Fibr3', 'Fibr1', 'Fibr4', 'Endo1', 'Epi1',
       'Peri1', 'NonImmune'], dtype=object)